# Flux Sweeping via SRS

[srsinst pacakge](https://github.com/thinkSRS/srsinst.dc205)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from flux_fit import *
from utils import *

Connect to the instruments

In [4]:
!python -m srsinst.dc205

In [5]:
client = Labber.connectToServer(timeout=None)
instrument_list = client.getListOfInstruments()
instruments = create_instrument_connections(client, instrument_list)

Successfully connected to Agilent Network Analyzer E5071B
Successfully connected to HP Spectrum Analyzer
Successfully connected to Keithley 2400 SourceMeter
Successfully connected to Manual
Successfully connected to Rohde&Schwarz RF Source
Successfully connected to SignalCore SC5511A Signal Generator_10002F25
Successfully connected to SignalCore SC5511A Signal Generator_1000334C
Successfully connected to Vaunix Lab Brick Digital Attenuator


In [6]:
vs = connect_SRS()
VNA = instruments['Agilent Network Analyzer E5071B']
DA = instruments['Vaunix Lab Brick Digital Attenuator']
SMU = instruments['Keithley 2400 SourceMeter']
LO= instruments['SignalCore SC5511A Signal Generator_10002F25']
Drive= instruments['SignalCore SC5511A Signal Generator_1000334C']
TWPA_PUMP = instruments['Rohde&Schwarz RF Source']

initialize_instruments(VNA, DA, SMU, LO, Drive, vs, TWPA_PUMP)

Since we only need the VNA and SRS for this measurement- lets make sure they are connected and ready to go. 

In [7]:
vs.check_id()


('DC205', 's/n20500691', 'ver1.80')

In [8]:
VNA.isRunning()

True

Setting up the paths for storing the Labber file.

In [ ]:
# get the time module
import time
import os
import logging

SPATH = r"G:\Shared drives\Labber\Data\{}\{}\Data_{}\\".format(time.strftime("%Y"), time.strftime("%m"), time.strftime("%m%d"))

figpath = SPATH + r"Figures\\"
if not os.path.exists(figpath):
    os.makedirs(figpath)
print(f"Saving path of Labber data to {SPATH}")
print(f"Saving figures to {figpath}")
logging.basicConfig(filename=SPATH+f'MEASUREMENTLOG_{time.strftime("%Y%m%d_%H%M%S")}.log',filemode='w',level=logging.INFO)


Either set the VNA settings manually and grab them.

In [26]:
VNA.getInstrConfig()

{'S11 - Enabled': False,
 'S21 - Enabled': True,
 'S12 - Enabled': False,
 'S22 - Enabled': False,
 'S11': {'y': array([], dtype=float64), 't0': 0.0, 'dt': 1.0, 'shape': [0]},
 'S21': {'y': array([0.01469547-0.00060081j, 0.01473039-0.00058496j,
         0.01471947-0.00060092j, ..., 0.01098988-0.00481419j,
         0.01098875-0.00480484j, 0.01100985-0.00484289j], dtype=complex64),
  't0': 5700000000.0,
  'dt': 62500.0,
  'shape': [1601]},
 'S12': {'y': array([], dtype=float64), 't0': 0.0, 'dt': 1.0, 'shape': [0]},
 'S22': {'y': array([], dtype=float64), 't0': 0.0, 'dt': 1.0, 'shape': [0]},
 'S11 - Value': 0j,
 'S21 - Value': 0j,
 'S12 - Value': 0j,
 'S22 - Value': 0j,
 'Output enabled': True,
 'Output power': 5.0,
 'Trigger': True,
 'IF bandwidth': 1000.0,
 'Average': True,
 '# of averages': 20.0,
 'Wait for new trace': True,
 'Electrical Delay': 8.2417e-08,
 'Phase Offset': 0.0,
 'Range type': 'Start - Stop',
 'Start frequency': 5700000000.0,
 'Stop frequency': 5800000000.0,
 '# of poi

Or just set them in the code. 

```python

In [ ]:
VNA.setValue('Output enabled',True)
VNA.setValue('Range type','Start - Stop')
VNA.setValue('Start frequency', 5.7E9)
VNA.setValue('Stop frequency', 5.8E9)
VNA.setValue('# of averages', 20)
VNA.setValue('Average',True)
VNA.setValue('Wait for new trace', True)

Testing the VNA settings by grabbing a trace

In [ ]:
dBG = VNA.getValue('S21')
xBG = np.arange(dBG['t0'],dBG['t0']+dBG['shape'][0]*dBG['dt'],dBG['dt'])
zBG = dBG['y']
plt.plot(zBG)
plt.show()

In [27]:
voltages = np.arange(-74E-3,72E-3,2E-3)
#voltages = np.arange(0,4E-3,2E-3)

# reverse order
#voltages = np.flip(voltages)
voltages, len(voltages)

(array([-7.4000000e-02, -7.2000000e-02, -7.0000000e-02, -6.8000000e-02,
        -6.6000000e-02, -6.4000000e-02, -6.2000000e-02, -6.0000000e-02,
        -5.8000000e-02, -5.6000000e-02, -5.4000000e-02, -5.2000000e-02,
        -5.0000000e-02, -4.8000000e-02, -4.6000000e-02, -4.4000000e-02,
        -4.2000000e-02, -4.0000000e-02, -3.8000000e-02, -3.6000000e-02,
        -3.4000000e-02, -3.2000000e-02, -3.0000000e-02, -2.8000000e-02,
        -2.6000000e-02, -2.4000000e-02, -2.2000000e-02, -2.0000000e-02,
        -1.8000000e-02, -1.6000000e-02, -1.4000000e-02, -1.2000000e-02,
        -1.0000000e-02, -8.0000000e-03, -6.0000000e-03, -4.0000000e-03,
        -2.0000000e-03,  6.9388939e-17,  2.0000000e-03,  4.0000000e-03,
         6.0000000e-03,  8.0000000e-03,  1.0000000e-02,  1.2000000e-02,
         1.4000000e-02,  1.6000000e-02,  1.8000000e-02,  2.0000000e-02,
         2.2000000e-02,  2.4000000e-02,  2.6000000e-02,  2.8000000e-02,
         3.0000000e-02,  3.2000000e-02,  3.4000000e-02,  3.60000

In [28]:
labber_file_name = "NBRL1A_70mK"

lfVNA = Labber.createLogFile_ForData(os.path.join(SPATH, f'{labber_file_name}_flux_sweep'),
                                      [{'name':'VNA - S21','complex':True,'vector':True,'x_name':'Frequency','x_unit':'Hz'}],
                                      step_channels=[{'name':'Voltage','values':voltages,'unit':'V'}])

In [29]:
for voltage in tqdm(voltages, desc='Voltage Sweep', unit='V'):  # Add tqdm progress bar
    print(voltage)
    logging.info(f'Starting Voltage = {voltage:.3f}\\n')
    set_flux_bias_srs(voltage, step=5E-5)
    sleep(0.05)
    dData = VNA.getValue('S21')
    zData = dData['y']
    xBG = np.arange(dData['t0'],dData['t0']+dData['shape'][0]*dData['dt'],dData['dt'])
    td2 = Labber.getTraceDict(zData,x0=xBG[0],x1=xBG[-1])
    lfVNA.addEntry({'VNA - S21':td2})

Voltage Sweep:   0%|          | 0/73 [00:00<?, ?V/s]

-0.074
Setting FFL bias to -74.0 mV from 0.0 mV


Voltage Sweep:   1%|▏         | 1/73 [13:10<15:48:47, 790.66s/V]

-0.072
Setting FFL bias to -72.0 mV from -74.0 mV


Voltage Sweep:   3%|▎         | 2/73 [14:04<7:02:58, 357.45s/V] 

...

Voltage Sweep:  97%|█████████▋| 71/73 [1:16:27<01:48, 54.24s/V]

0.06800000000000013
Setting FFL bias to 68.00000000000013 mV from 66.0 mV


Voltage Sweep:  99%|█████████▊| 72/73 [1:17:21<00:54, 54.23s/V]

0.07000000000000013
Setting FFL bias to 70.00000000000013 mV from 68.0 mV


Voltage Sweep: 100%|██████████| 73/73 [1:18:15<00:00, 64.33s/V]


Ensure that the flux bias is set back to zero.

In [9]:
set_flux_bias_srs(0, step=5E-5)

Setting FFL bias to 0.0 mV from 18.9 mV
